In [3]:
!pip install scikit-learn
!pip install xlwt xlrd xlutils
!pip install matplotlib
!pip install opencv-python


In [12]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

# Path to dataset
DATASET_PATH = 'dataset'

# Initialize data and labels
data = []
labels = []

# Create a list to store user IDs
user_ids = []

# Load images and labels
for image_name in os.listdir(DATASET_PATH):
    image_path = os.path.join(DATASET_PATH, image_name)
    
    # Only process images that match the pattern (e.g., User.1.1.jpg)
    if image_name.startswith("User") and image_name.endswith(".jpg"):
        # Extract the user ID from the filename (e.g., "User.1.1.jpg" → "1")
        user_id = image_name.split('.')[1]
        
        # Add user ID to the list if not already present
        if user_id not in user_ids:
            user_ids.append(user_id)
        
        # Get the label index (based on the position in the user_ids list)
        label_index = user_ids.index(user_id)
        
        # Load image and convert it to grayscale
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        
        # Repeat the grayscale image across 3 channels to make it (224, 224, 3)
        image = np.repeat(image[..., np.newaxis], 3, axis=-1)
        
        # Resize to (224, 224) as required by MobileNetV2
        image = cv2.resize(image, (224, 224))
        
        # Convert image to array
        image = img_to_array(image)

        # Append image data and label
        data.append(image)
        labels.append(label_index)

# Convert to NumPy arrays and normalize
data = np.array(data, dtype='float32') / 255.0
labels = np.array(labels)

# One-hot encode labels
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes)

# Split dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Debugging output
print(f"Total images: {len(data)}")
print(f"Number of unique users (classes): {num_classes}")
print(f"User IDs mapped to labels: {user_ids}")


Total images: 200
Number of unique users (classes): 4
User IDs mapped to labels: ['3', '4', '1', '2']


In [16]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load pre-trained MobileNetV2 model without the top (classification) layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Create a Sequential model
model = Sequential()

# Add the base pre-trained model
model.add(base_model)

# Flatten the output of the base model
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(128, activation='relu'))

# Add a dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Add the final output layer for classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Debugging output
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_5 (Flatten)         (None, 62720)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               8028288   
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 4)                 516       
                                                                 
Total params: 10,286,788
Trainable params: 8,028,804
Non-trainable params: 2,257,984
___________________________________

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_test, y_test),
    callbacks=[early_stop]
)


Epoch 1/10
5/5 [==============================] - 3s 336ms/step - loss: 0.8308 - accuracy: 0.7000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 1s 289ms/step - loss: 0.0514 - accuracy: 0.9812 - val_loss: 1.2441e-04 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 1s 251ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 2.1269e-05 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 1s 241ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 7.2863e-06 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 1s 239ms/step - loss: 3.9605e-04 - accuracy: 1.0000 - val_loss: 3.7193e-06 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 1s 238ms/step - loss: 2.9041e-04 - accuracy: 1.0000 - val_loss: 2.6404e-06 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 1s 233ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 4.5805e-06 - val_accur

In [20]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

model.save("face_recognition_model.h5")


2/2 [==============================] - 0s 66ms/step - loss: 1.7524e-06 - accuracy: 1.0000
Test Accuracy: 100.00%


In [3]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from datetime import datetime
import xlwt
from xlrd import open_workbook
from xlutils.copy import copy
from pathlib import Path


# Define the output function for recording attendance
def output(filename, sheet, num, name, present):
    # Ensure the directory exists
    Path("attendance_sheet").mkdir(exist_ok=True)

    # Define the file path
    file_path = Path(f"attendance_sheet/{filename}{datetime.now().date()}.xls")

    if file_path.is_file():
        # Open the existing file
        rb = open_workbook(file_path)
        book = copy(rb)
        sh = book.get_sheet(0)

        # Check if the date is already written in the first cell
        try:
            first_cell_value = rb.sheet_by_index(0).cell_value(0, 0)
        except IndexError:
            first_cell_value = None
    else:
        # Create a new workbook and sheet
        book = xlwt.Workbook()
        sh = book.add_sheet(sheet)
        first_cell_value = None

        # Add headers for new sheets
        style_header = xlwt.easyxf('font: name Times New Roman, color-index red, bold on')
        sh.write(1, 0, "Name", style_header)
        sh.write(1, 1, "Present", style_header)

    # Write the date if not already written
    if not first_cell_value:
        style_date = xlwt.easyxf(num_format_str='D-MMM-YY')
        sh.write(0, 0, datetime.now().date(), style_date)

    # Check for duplicates in the attendance list
    if file_path.is_file():
        rb_sheet = rb.sheet_by_index(0)
        existing_names = [rb_sheet.cell_value(row, 0) for row in range(2, rb_sheet.nrows)]
    else:
        existing_names = []

    if name not in existing_names:
        # Write attendance data
        sh.write(num + 1, 0, name)
        sh.write(num + 1, 1, present)

    # Save the workbook
    fullname = file_path.as_posix()
    book.save(fullname)
    return fullname


# Load the trained model
model = load_model('face_recognition_model.h5')

# Load Haarcascade classifier for face detection
face_cascade = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')

# Label names (matching the user_id order)
label_names = ['Tanzim', 'Nayeem', 'Raiyan', 'Alvee']  # Correct mapping based on your folder structure
user_id_map = {'Tanzim': 1, 'Nayeem': 2, 'Raiyan': 3, 'Alvee': 4}  # Map names to user IDs

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Counter for attendance row in Excel
num = 2  # Start from row 2 to avoid overwriting headers

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # List to store face regions and bounding boxes
    face_boxes = []
    face_regions = []

    # Collect bounding boxes and regions
    for (x, y, w, h) in faces:
        face_boxes.append((x, y, x + w, y + h))
        roi = gray[y:y + h, x:x + w]
        roi = cv2.resize(roi, (224, 224))  # Resize to (224, 224)
        roi = cv2.cvtColor(roi, cv2.COLOR_GRAY2RGB)  # Convert to RGB
        roi = roi.astype('float32') / 255.0  # Normalize
        roi = np.expand_dims(roi, axis=0)  # Add batch dimension


    # Ensure the number of faces matches the number of predictions
    for i, (box, roi) in enumerate(zip(face_boxes, face_regions)):
        (x1, y1, x2, y2) = box

        # Predict the face
        preds = model.predict(roi)
        predicted_class = np.argmax(preds)
        confidence = np.max(preds) * 100

        # Handle out-of-range predictions
        if predicted_class < len(label_names):
            label = label_names[predicted_class]
        else:
            label = "Unknown"

        # Draw rectangle and label on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label}: {confidence:.2f}%", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Mark attendance in the Excel file
        if label != "Unknown":
            output('attendance_', 'Sheet1', num, label, "Present")
            num += 1  # Move to the next row for each recognized person

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


2025-02-03 18:31:12.710 python[8872:167571] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-03 18:31:12.710 python[8872:167571] +[IMKInputSession subclass]: chose IMKInputSession_Modern
